In [1]:
import os
import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Polygon, Point

# ---------------------------
# CONFIG
# ---------------------------
RAW_DIR = "../../raw/osm"
OUT_DIR = "../../processed/networks"
os.makedirs(OUT_DIR, exist_ok=True)

JARNTORGET_CENTER = (57.699972, 11.952843)
RADIUS_METERS = 3000
TARGET_CRS = "EPSG:3006"  # SWEREF99 TM

# ---------------------------
# SQUARE BOUNDARY & EXCLUSION POLYGON
# ---------------------------
center_lon, center_lat = JARNTORGET_CENTER[1], JARNTORGET_CENTER[0]
radius_deg = RADIUS_METERS / 111320  # approx. conversion
square_boundary = Polygon([
    (center_lon - radius_deg, center_lat - radius_deg),
    (center_lon + radius_deg, center_lat - radius_deg),
    (center_lon + radius_deg, center_lat + radius_deg),
    (center_lon - radius_deg, center_lat + radius_deg)
])

# River exclusion
exclusion_polygon = Polygon([
    (11.919817, 57.699166),
    (11.948687, 57.703883),  
    (11.956237, 57.709494),
    (11.967546, 57.715251),
    (11.981203, 57.720014),
    (11.990000, 57.730000),
    (11.900000, 57.730000),
    (11.919817, 57.699166)
])

# ---------------------------
# CONFIGURE OSMNX
# ---------------------------
ox.settings.log_console = True
ox.settings.use_cache = True

# ---------------------------
# DOWNLOAD NETWORKS
# ---------------------------
G_drive = ox.graph_from_point(JARNTORGET_CENTER, dist=RADIUS_METERS*1.5, network_type='drive')
G_walk = ox.graph_from_point(JARNTORGET_CENTER, dist=RADIUS_METERS*1.5, network_type='walk')


# ---------------------------
# ADD EDGE ATTRIBUTES
# ---------------------------
G_drive = ox.add_edge_speeds(G_drive)
G_drive = ox.add_edge_travel_times(G_drive)

# ---------------------------
# CLIP GRAPHS TO SQUARE
# ---------------------------
G_drive = ox.truncate.truncate_graph_polygon(G_drive, square_boundary)
G_walk = ox.truncate.truncate_graph_polygon(G_walk, square_boundary)

# ---------------------------
# REMOVE EDGES OVER RIVER
# ---------------------------
def remove_edges_over_polygon(G, polygon):
    G_clean = G.copy()
    to_remove = []
    for u, v, k, data in G_clean.edges(keys=True, data=True):
        geom = data.get("geometry")
        if geom is None:
            continue
        centroid = geom.centroid if geom.geom_type != 'Point' else geom
        if polygon.contains(centroid):
            to_remove.append((u, v, k))
    G_clean.remove_edges_from(to_remove)
    # Remove isolated nodes
    isolated = list(nx.isolates(G_clean))
    G_clean.remove_nodes_from(isolated)
    return G_clean

G_drive = remove_edges_over_polygon(G_drive, exclusion_polygon)
G_walk = remove_edges_over_polygon(G_walk, exclusion_polygon)

def keep_largest_component(G, strongly=False):
    if strongly:
        # largest strongly connected component
        largest_nodes = max(nx.strongly_connected_components(G), key=len)
    else:
        # convert to undirected first
        G_undirected = G.to_undirected()
        largest_nodes = max(nx.connected_components(G_undirected), key=len)
    return G.subgraph(largest_nodes).copy()


G_drive = keep_largest_component(G_drive, strongly=True)
G_walk = keep_largest_component(G_walk, strongly=False)



# ---------------------------
# CONVERT TO GEODATAFRAMES
# ---------------------------
drive_nodes, drive_edges = ox.graph_to_gdfs(G_drive)
walk_nodes, walk_edges = ox.graph_to_gdfs(G_walk)


# ---------------------------
# SAVE DATA
# ---------------------------
drive_nodes.to_file(os.path.join(OUT_DIR, "drive_nodes_clean.gpkg"), driver="GPKG")
drive_edges.to_file(os.path.join(OUT_DIR, "drive_edges_clean.gpkg"), driver="GPKG")

walk_nodes.to_file(os.path.join(OUT_DIR, "walk_nodes_clean.gpkg"), driver="GPKG")
walk_edges.to_file(os.path.join(OUT_DIR, "walk_edges_clean.gpkg"), driver="GPKG")


# ---------------------------
# SUMMARY
# ---------------------------
print("=== Preprocessing Complete ===")
print(f"Drive: {G_drive.number_of_nodes()} nodes, {G_drive.number_of_edges()} edges")
print(f"Pedestrian: {G_walk.number_of_nodes()} nodes, {G_walk.number_of_edges()} edges")


=== Preprocessing Complete ===
Drive: 1171 nodes, 2414 edges
Pedestrian: 14194 nodes, 37660 edges


In [2]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

print("Extracting Göta Älv by geometry (area-based, with reprojection)…")

# Step 1: query all water polygons
layers = []

try:
    wr = ox.features_from_place("Göteborg, Sweden", tags={"water": "river"})
    layers.append(wr)
    print(f"water=river: {len(wr)} features")
except:
    print("water=river failed")

try:
    nw = ox.features_from_place("Göteborg, Sweden", tags={"natural": "water"})
    layers.append(nw)
    print(f"natural=water: {len(nw)} features")
except:
    print("natural=water failed")


# Merge
merged = gpd.GeoDataFrame(pd.concat(layers, ignore_index=True),
                          crs=layers[0].crs)

# Keep only polygons
merged = merged[merged.geometry.type.isin(["Polygon", "MultiPolygon"])]

# -----------------------------------------------------------
# NEW: Reproject to Swedish CRS (meters) for area calculation
# -----------------------------------------------------------
SWEDEN_EPSG = 3006
epsg_3006 = 3006
merged_m = merged.to_crs(SWEDEN_EPSG)

# Compute area in m²
merged_m["area"] = merged_m.area

# Select the largest polygons
river = merged_m.sort_values("area", ascending=False).head(2)

print("Largest polygons by area (m²):")
print(river[["area"]])

# Reproject back to WGS84 for saving and plotting
river = river.to_crs(4326)

# Save
OUT_DIR = "../../processed/networks"
river_file = f"{OUT_DIR}/gota_alv.gpkg"
river.to_file(river_file, driver="GPKG")

print("Saved Göta Älv polygons to:", river_file)


Extracting Göta Älv by geometry (area-based, with reprojection)…
water=river: 16 features
natural=water: 831 features
Largest polygons by area (m²):
            area
44  7.590459e+06
4   7.590459e+06
Saved Göta Älv polygons to: ../../processed/networks/gota_alv.gpkg
